In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Resnet50 
# Optimizer Adam , Learning Rate=.0001, Downsampled dataset.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import unicodedata
import matplotlib.pyplot as plt
import seaborn as sns
import os, re
from pprint import pprint
import matplotlib.patches as patches
from matplotlib.cbook import print_cycles
from PIL import Image
import glob
import cv2
from skimage import color
from keras.datasets import mnist

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop, Adam
import numpy as np
import os

In [ ]:
from pathlib import Path

In [ ]:
# Paths to the base directories/files of the dataset
base_dir = Path('/kaggle/input/cassava-leaf-disease-classification/')
data  = pd.read_csv(f'{base_dir}/train.csv')

In [ ]:
data

In [ ]:
data2=data.copy()

In [ ]:
data2.head()

# Data Preprocessiong

In [ ]:
label_dict = {
0:"Cassava Bacterial Blight (CBB)",
1:"Cassava Brown Streak Disease (CBSD)",
2:"Cassava Green Mottle (CGM)",
3:"Cassava Mosaic Disease (CMD)",
4:"Healthy"
}
label_dict

In [ ]:
data['label'].replace(label_dict, inplace=True)

In [ ]:
data.head()

In [ ]:
data2.head()

In [ ]:
from PIL import Image

# Downsampling dataset

In [ ]:
mask = data2['label'] ==4
classHealthy = data[mask]

In [ ]:
mask = data2['label']==3
classCMD = data[mask]

In [ ]:
mask = data2['label'] ==2
classCGM  = data[mask]

In [ ]:
mask = data2['label'] ==1
classCBSD  = data[mask]

In [ ]:
mask = data2['label'] ==0
classCBB  = data[mask]

In [ ]:
data2['label'].hist()

In [ ]:
class0 = classCBB.sample(frac=0.99)
class1 = classCBSD.sample(frac=0.99)
class2 = classCGM.sample(frac=0.99)
class3 = classCMD.sample(frac=0.2)
class4 = classHealthy.sample(frac=0.99)




frames=[class0,class1,class2,class3,class4]
scaled_Data = pd.concat(frames)
#finalData.head(10)
print(len(scaled_Data))

In [ ]:
scaled_Data['label'].hist()

In [ ]:
scaled_Data.head(10)

# Dataset split

In [ ]:
from sklearn.model_selection import train_test_split

train,valid = train_test_split(scaled_Data, test_size = 0.2, random_state = 42, stratify = scaled_Data.label)

In [ ]:
valid.head()

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.0)
train_data = train_datagen.flow_from_dataframe(train ,directory='/kaggle/input/cassava-leaf-disease-classification/train_images',
                                                    x_col="image_id" , y_col="label", target_size=(128, 128), color_mode="rgb",
                                                    class_mode="categorical",batch_size=128)
valid_data = train_datagen.flow_from_dataframe(valid ,directory='/kaggle/input/cassava-leaf-disease-classification/train_images',
                                                    x_col="image_id" , y_col="label", target_size=(128, 128), color_mode="rgb",
                                                    class_mode="categorical",batch_size=32)

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (128,128,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    keras.layers.Dense(64, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)),
    #model.add(Dropout(0.7))
    keras.layers.Dense(64, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dense(512,activation = 'relu'),
    tf.keras.layers.Dense(512,activation = 'relu'),
    tf.keras.layers.Dense(5,activation = 'softmax')
])
model.compile(optimizer=RMSprop(learning_rate=0.0005),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=8,
    restore_best_weights=True
)

# Model train

In [ ]:
history=model.fit(train_data,
          epochs = 20,
          validation_data = valid_data,
          callbacks = [callback],verbose = 1)

In [ ]:
model.save('cassava-model.h5')

In [ ]:
est_loss, test_acc = model.evaluate(valid_data, verbose=2) 

print('Test accuracy:', test_acc)

In [ ]:
fig, axs = plt.subplots(nrows = 1, ncols =2 , figsize = (10,5))
axs[0].plot(range(len(history.history['accuracy'])), history.history['accuracy'], linewidth = 2, label = "training")
axs[0].plot(range(len(history.history['accuracy'])), history.history['val_accuracy'], linewidth = 2, label = "validation")
axs[1].plot(range(len(history.history['loss'])), history.history['loss'], linewidth = 2, label = "training")
axs[1].plot(range(len(history.history['loss'])), history.history['val_loss'], linewidth = 2, label = "validation")
axs[0].set_xlabel("epochs")
axs[0].set_xlabel("epochs")
axs[0].set_ylabel("accuracy")
axs[1].set_ylabel("loss")
plt.legend()

In [ ]:
from keras.preprocessing import image
test_image = image.load_img('/kaggle/input/cassava-leaf-disease-classification/test_images/2216849948.jpg', 
                            target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

In [ ]:
submission_data = pd.DataFrame(columns = ['image_id','label'])
submission_data['image_id'] = os.listdir('/kaggle/input/cassava-leaf-disease-classification/test_images')
submission_data['label'] = np.argmax(result)

In [ ]:
submission_data.to_csv('submission.csv',index=False)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

#num_of_test_samples = 3209
#batch_size = 1
steps = valid_data.samples/valid_data.batch_size
predictions = model.predict_generator(valid_data, steps)

#Y_pred = loaded_model.predict_generator(valid_data, valid_data.samples // valid_data.batch_size+5)
#y_pred = np.argmax(Y_pred, axis=1)
y_pred = np.argmax(predictions, axis=1)
print('Confusion Matrix')
print(confusion_matrix(valid_data.classes, y_pred))

In [ ]:
target_names = list(train_data.class_indices.keys()) # Classes
print(classification_report(valid_data.classes, y_pred, target_names=target_names))

In [ ]:
import seaborn as sns

cm = confusion_matrix(valid_data.classes, y_pred)
labels = ['Cassava Bacterial Blight (CBB)', 'Cassava Brown Streak Disease (CBSD)', 'Cassava Green Mottle (CGM)', 'Cassava Mosaic Disease (CMD)','Healthy']
plt.figure(figsize=(8,6))
sns.heatmap(cm,xticklabels=labels, yticklabels=labels, annot=True, fmt='d', cmap="Blues", vmin = 0.2);
plt.title('Confusion Matrix')
plt.ylabel('True Class')
plt.xlabel('Predicted Class')
plt.show()